<h1><center>Basic Bioinformatics Workflows on OSCAR: Snakemake and Nextflow</center></h1>
<p><center>Instructors: Ashok Ragavendran and Jordan Lawson</center>
 <center>Center for Computation and Visualization</center>
 <center>Center for Computational Biology of Human Disease - Computational Biology Core</center></p>

## What is Snakemake and Nextflow?  

Snakemake and Nextflow are workflow management tools that allow users to easily write data-intensive computational **pipelines**. These pipelines, or workflows as they are also called, have the following key features:

- Sequential processing of files
- Usually requires more than one tool
- Multiple programming languages
- Most times each sample is processed individually
- Compute resource intensive
  - Alignment could take 16 cpus, 60 Gb RAM, 4-24 hours, 30Gb of disk space per sample

## Why do we care about these pipelines? 

### Reason 1: Reproducibility 

<img src="./img/reproduce.png" width="700"/>

The journal Nature published a survey that found that more than 70% of researchers have tried and failed to reproduce another scientist's experiments. This trend is hugely problematic because we then can't trust the findings from many studies enough to use them to make data-driven decisions. In short, we need tools and standards that help address the reproducibility crisis in science! 

Pipelines created with Snakemake and Nextflow incorporate version-control and state-of-the-art software tools, known as containers, to manage all software dependencies and create stable environments that can be trusted to reproduce analyses reliably and accurately. 

***Reproducibility is important for producing trustworthy research!***

### Reason 2: Portability

#### What if we need to perform analyses with more resources?

This type of scenario would require us to move our analyses to a different environment, for example, a High Performance Computing (HPC) cluster environment. 

An important feature of Snakemake and Nextflow workflow management tools is that they enable users to easily scale any pipeline written on a personal computer to then run on an HPC cluster such as OSCAR, the HPC cluster we use at Brown University. So now we can run our pipelines using high performance resources without having to change workflow definitions or hard-code a pipeline to a specific setup. As a result, **the code stays constant** across varying infrastructures, thereby allowing portability, easy collaboration, and avoiding lock-in. 

***In short, we can easily move our multi-step analyses (i.e., pipelines) to any place we need them!***

## So Let's See How All This Works! 

### Our Starting Point

Say we have samples from reduced representation bisulfite sequencing (RRBS data) that we need to process on OSCAR by performing the following set of actions: 

<img src="./img/workflow.png" width="700"/>

<h2><center>What do you do??</center></h2>

## A Naive Approach

One solution would be to write a bunch of shell scripts that use various software tools to process the data in the ways we need. 

For example, if we need to run fastqc, trim galore, and then an alignment, we could just write a shell script for each step - so a total of 4 shell scripts in this case - where we have various inputs and outputs. This would look something as follows: 

**Script 1: Fastqc**

```
#!/bin/bash
#SBATCH -t 48:00:00
#SBATCH -n 32
#SBATCH -J rrbs_fastqc
#SBATCH --mem=198GB
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jordan_lawson@brown.edu

source /gpfs/runtime/cbc_conda/bin/activate_cbc_conda
conda activate fedulov_rrbs
for sample in `ls /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/Sequencing_Files/*fastq.gz`
do
align_dir="/gpfs/data/cbc/fedulov_alexey/porcine_rrbs" 
fastqc -o ${align_dir}/fastqc $sample
done
```

**Script 2: Trimming** 

```
#!/bin/bash
#SBATCH -t 48:00:00
#SBATCH -n 32
#SBATCH -J trimmomatic_update
#SBATCH --mem=198GB
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jordan_lawson@brown.edu

source /gpfs/runtime/cbc_conda/bin/activate_cbc_conda

for sample in `ls /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/trim_galore/*_trimmed.fq.gz`
do
    dir="/gpfs/data/cbc/fedulov_alexey/porcine_rrbs/trimmomatic"
    base=$(basename $sample "_trimmed.fq.gz")
    trimmomatic SE  -threads 8 -trimlog ${dir}/${base}_SE.log $sample ${dir}/${base}_tr.fq.gz ILLUMINACLIP:/gpfs/data/cbc/cbc_conda_v1/envs/cbc_conda/opt/trimmomatic-0.36/adapters/TruSeq3-SE.fa:2:30:5:6:true SLIDINGWINDOW:4:20 MINLEN:35
done 
```


**Script 3: Fastqc on trimmed reads**

```
#!/bin/bash
#SBATCH -t 24:00:00
#SBATCH -n 8
#SBATCH -J retrim_fastqc_update
#SBATCH --mem=16GB
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jordan_lawson@brown.edu

source /gpfs/runtime/cbc_conda/bin/activate_cbc_conda
conda activate fedulov_rrbs
for sample in `ls /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/trimmomatic/*_tr.fq.gz`
do
trim_qc_dir="/gpfs/data/cbc/fedulov_alexey/porcine_rrbs"
fastqc -o ${trim_qc_dir}/trimmomatic_qc $sample
done
```

**Script 4: Alignment**

```
#!/bin/bash
#SBATCH -t 24:00:00
#SBATCH -N 1
#SBATCH -n 16
#SBATCH -J bismark_align_update_redo
#SBATCH --mem=160GB
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jordan_lawson@brown.edu
#SBATCH --array=1-18
#SBATCH -e /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/logs/bismark_align_%a_%A_%j.err
#SBATCH -o /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/logs/bismark_align_%a_%A_%j.out

source /gpfs/runtime/cbc_conda/bin/activate_cbc_conda
conda activate fedulov_rrbs
input=($(ls /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/trimmomatic/*_tr.fq.gz)) # using the round brackets indicates that this is a bash array
bismark -o /gpfs/data/cbc/fedulov_alexey/porcine_rrbs/alignments --bowtie2 --genome /gpfs/data/shared/databases/refchef_refs/S_scrofa/primary/bismark_index --un --pbat ${input[$((SLURM_ARRAY_TASK_ID -1))]}
```

## Problems with the Naive Approach 

Using multiple shell scripts to create a makeshift pipeline will work, but it is **inefficient**, can **get complicated fast**, and there are a few **challenges you have to manage**, such as: 

* Making sure you have the appropriate software and all dependencies for each step in the analysis - this can be a lot to stay on top of if you have a pipeline with a lot of steps! (imagine a 10 step process)
* **Portability!** Building and running on different machines is much more work
* Specifying where your output will go 
* Calling in the appropriate input (which is often the output from a previous step) 
* Handling where log files go 
* More labor intensive - we have to stay on top of jobs and monitor when each step finishes and then run next

## A Smarter Approach: Using Workflow Managers! 

The solution for processing your data in a much more efficient manner that handles the aforementioned issues is workflow management tools, such as Snakemake and Nextflow. Let's now learn how to use Snakemake and Nextflow...

## Tutorial: Using workflow management tools on OSCAR 

Workflow management tools are software that allow you to write more efficient, portable computational pipelines. As a result, you are able to optimize your workflows while maintaining reproducibility and rigor. Note that there are many workflow management tools available to researchers, but the two tools we will focus on learning about today are **Snakemake** and **Nextflow**. Let's first start with Snakemake...

<img src="./img/snakemake.png" width="500"/>

### Step 1: The Setup

Let's first discuss setting up our environment on OSCAR so that we can get Snakemake up and running. There are a few ways to run Snakemake on OSCAR. For example, one could set up Snakemake through Conda environments or you could use singularity containers. The details of these tools (i.e., Conda environments and singularity containers) are beyond the scope of this workshop, but here are some helpful links to get you started learning more about these tools, should you be interested: 

<u>Conda and Conda environments:</u> 

https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html <br>

https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html <br>


<u> Singularity containers: </u> 

https://sylabs.io/guides/3.5/user-guide/introduction.html <br>

https://blogs.iu.edu/ncgas/2021/04/29/a-quick-intro-to-singularity-containers/ <br>


For this workshop, the specific approach we are going to take is to set up a Python virtual environment (essentially just an isolated environment containing the few pieces of software we need to get up and running) and then within this environment we tell Snakemake to download and run specific pre-built singularity containers for each of the different data processing steps (known as Snakemake rules). Singularity containers are just computer programs a.k.a virtual machines that encapsulate all the software needed for a workflow and thus enable reproducibility. They are widely used on HPC systems because of their increased security relative to other software options.

So now let's get started with our set up. First, ssh into OSCAR and wherever you like (and have enough space for storage) create a folder called **snakemake**. To do this, simply type ```mkdir snakemake```. 

Now enter this folder using ```cd /path/to/snakemake``` and inside it we will set up our virtual environment so that we can have access to and run Snakemake software on OSCAR. To set up our virtual environment, we use the following script: 

<br> 

```
module load python/3.9.0    # load version of Python needed 
virtualenv snakemake_env     # create environment 
source snakemake_env/bin/activate    # activate environment
pip3 install snakemake    # install snakemake using pip
deactivate    # deactivate and exit, we will use again later!
```
    
**Note:** Save and run this script in your snakemake folder using ```bash env.sh```. This will create a snakemake_env folder in your project directory. 

After this code has been run, you can now at any point type ```source /path/to/environment/environment_folder_name/bin/activate``` and you will enter an environment that has snakemake software installed and ready to be called for use. Now let's move onto setting up the specific pieces of input that Snakemake needs to run on OSCAR, our HPC cluster. 

**At this point, I am also going to open my terminal on Open OnDemand (OOD) so that I can walk you through and show you how each of these steps and files below look. Feel free to open your terminal as well and follow along. To do so, you can go to OOD at https://ood.ccv.brown.edu/pun/sys/dashboard and under the Clusters tab at the top select >_OSCAR Shell Access.**

### Step 2: Creating the Snakefile 

To start using Snakemake on any platform, the first thing one must do is create a **Snakefile**. A Snakefile is a file that defines a Snakemake workflow in terms of rules that are to be carried out in a specific order to complete a pipeline. This file determines the entire flow of your data analysis pipeline, specifying the rules to be carried out and their respective inputs and outputs. We can name this file whatever we like, as long as we tell our Snakemake program where to find it; however, by convention, we usually name this file Snakefile (with no extension!). 

I create a file called Snakefile and store this file in my newly created **snakemake** folder. Drawing on the same RRBS example we started with, the Snakefile would be: 

**Snakefile**

```python
## Snakefile for BootCamp ##

# Specify configuration file to use - optional
# configfile: "/path/to/file/config.yaml"

# Define sample to iterate across data 
sample=["sample_1", "sample_2", "sample_3", "sample_4"]

# Create rules
rule all:
    message: "All done....!"
    input:
        expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_trimmed.fq.gz", sample=sample),
        expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}.fastq.gz_trimming_report.txt", sample=sample),
        expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_bismark_bt2_pe.bam", sample=sample),
        expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_bismark_bt2_PE_report.txt", sample=sample),
        expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_bismark_bt2_pe.nucleotide_stats.txt", sample=sample)


rule fastqc:
    message: "Running FastQC..."
    input:
        rawread=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/data_new/{sample}.fq.gz", sample=sample) 
    output:
        zip=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/fastqc/{sample}.zip", sample=sample), 
        html=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/fastqc/{sample}.html", sample=sample)
    singularity: "library://ftabaro/default/methylsnake" 
    shell: "fastqc {input.rawread} -o /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/fastqc"

rule trim:
   message: "Performing reads trimming..." 
   input:
       rawreads=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/data_new/{sample}.fq.gz", sample=sample)
   output:
       trimmed=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_trimmed.fq.gz", sample=sample),
       report=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}.fastq.gz_trimming_report.txt", sample=sample)
   params:  
     quality_filter_value="22",
   threads: 4
   singularity: "library://ftabaro/default/methylsnake"
   shell: "trim_galore --quality {params.quality_filter_value} --phred33 --output_dir /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim --gzip --rrbs --fastqc --cores {threads} {input.rawreads}" 

rule bismark_align:
    message: "Performing alignment..."
    input:
        trimreads=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_trimmed.fq.gz", sample=sample)
    output: 
      bam=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_bismark_bt2_pe.bam", sample=sample),
      bisreport=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_bismark_bt2_PE_report.txt", sample=sample),
      stats=expand("/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/trim/{sample}_bismark_bt2_pe.nucleotide_stats.txt", sample=sample)
    threads: 4
    singularity: "library://ftabaro/default/methylsnake"
    shell: "bismark -o /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/alignment --bowtie2 --genome /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/index --un --pbat {input.trimreads}"
```

### Step 3: Telling Snakemake how to run on the HPC Cluster  

After defining the rules and analysis steps for our workflow via the Snakefile, we now must set up the workflow to be compatible with our HPC cluster, telling it how to assign resources to each rule. This is done by creating a yaml file called **cluster.yaml** (really it can be called anything, as long as the extension is .yaml). This file can be stored anywhere, as long as you specify where it can be found by the Snakemake program when it's run (will see this later). But for simplicity, it's best to just create and save it in the same place as your Snakefile. We specify the cluster.yaml file as follows: 

**cluster.yaml**

```yaml
__default__:
  partition: "batch"
  cpus: "1"
  time: 60
  mem: "4g"
  email: jordan_lawson@brown.edu  
  email_type: "ALL"
```

In the above, we create a default specification, where each rule, by default, will get this resource allocation when run. 

### Step 4: Bringing it all together with Shell Scripting 

Now here is the step where we tie everything together, running our Snakefile and cluster specification all with one script, a shell script. To run everything on OSCAR, we can use the following script: 

```
#!/bin/bash

##############################
#                             # 
#       Snakemake             #
#                             #
###############################

##### 1.) Job Sumbission Options ######

# Change these as needed 

#SBATCH -t 24:00:00
#SBATCH -n 2
#SBATCH -J snake_test
#SBATCH --mem=16GB
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jordan_lawson@brown.edu

###### 2.) Run snakemake #####

# Activate virtual environment 
source snakemake_env/bin/activate

# Run snakemake - note you will need a cluster.yaml file as this command references one!
snakemake --use-singularity --singularity-args "-B /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/:/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/" -s /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/Snakefile --cluster-config /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/snakemake/cluster.yaml --latency-wait 60 --cluster 'sbatch -t {cluster.time} --mem={cluster.mem} -c {cluster.cpus} --mail-type {cluster.email_type} --mail-user {cluster.email}' -j 10
```

I save this script as **snakemake.sh** and place it in the snakemake folder with the other files I recently created. Once we have this with everything else, we can run everything on the cluster by typing ```sbatch snakemake.sh```

Note in the above that we are running the snakemake program with the Snakefile and cluster.yaml files we specified. Some other important things: 

- your log file for slurm will automtatically be placed in the same directory that you ran the snakemake.sh script from. However, you can also get more detailed log files for each snakemake rule and its respective outputs by using the ```log:``` command followed by the path you want the logs files to go to in each of the rules found within the Snakefile 

- we are using the --use-singularity argument in the above shell script to allow for Snakemake to use a singularity container when running pipelines; this is highly recommended for reproducibility. 

- when using the singularity argument, **it is very, very important** that you include the ```--singularity-args "-B folder_to_mount:source_destination"``` argument or else your pipeline will fail to recognize any inputs and outputs you are specifying in the workflow 

## Next up..

<img src="./img/nextflow.png" width="500"/>

### Step 1: The Setup

Once again, our starting point is setting up our environment on OSCAR so that we can get Nextflow up and running. Like with Snakemake, there are a few ways to run Nextflow on OSCAR, but we will be using a similar approach to the one we used with Snakemake, setting up a Python virtual environment first so that we have access to the Nextflow program and then running pipelines from there.  

**Nextflow vs Snakemake:** Nextflow is very much like Snakemake in that they both serve the same need, so which one you use is largely up to you and is preference-based. However, one very notable difference is that Nextflow has an open-source, community built repository of bioinformatics pipelines that you can easily download and use for your own data processing needs so that you don't have to write your own pipelines (but you can if you want to!). This resource is called **nf-core** and is a community effort to collect a curated set of pipelines built using Nextflow. This is nice because it already has many of the workflows and analyses that computational biologists use and so we don't have to waste our efforts and time re-inventing the wheel! Given these advantages and our focus on bioinformatics, today's example will illustrate how to use Nextflow on OSCAR with nf-core. Note, however, that much of what we will learn today about using Nextflow with nf-core still applies to developing and running your own workflows with Nextflow without the use of nf-core. To learn more about nf-core, you can visit its homepage here: 

https://nf-co.re

So now let's get started with our set up. First, ssh into OSCAR and wherever you like (and have enough space for storage) create a folder called **nextflow**. To do this, simply type ```mkdir nextflow```. 

Now enter this folder using ```cd /path/to/nextflow``` and inside it we will set up our virtual environment so that we can have access to and run Nextflow software on OSCAR. To set up our virtual environment, we use the following script: 

**env.sh**

```
# Create virutal environment 
virtualenv nextflow_env 
source ~/nextflow_env/bin/activate 
wget -qO- https://get.nextflow.io | bash
# Make executable 
chmod +x nextflow
mv nextflow $HOME/nextflow_env/bin
deactivate
# You can now run nextflow in bash script using `source path/to/nextflow_env/bin/activate` 
```

**Note:** Save and run this script in your nextflow folder using ```bash env.sh```. This will create a nextflow_env folder in your project directory. 

###  Step 2: Configure Nextflow for HPC cluster 

Next, we tell Nextflow how to run pipelines on our HPC cluster by creating a **.config** file. We will call this .config file **nextflow.config** and store it inside or newly created nextflow folder. We create the file as follows: 

```
params {
  config_profile_description = 'CBC cluster profile'
  config_profile_contact = 'Jordan Lawson (jordan_lawson@brown.edu)'
  singularity_cache_dir = "/users/jlawson9/scratch"
  max_memory = 192.GB
  max_cpus = 56
  max_time = 24.h
}
singularity {
  enabled = true
  cacheDir=params.singularity_cache_dir
  autoMounts = true
}
process {
  executor = 'slurm'
  clusterOptions = '-p batch'
}
```


### Step 3: Giving nf-core Access to GitHub 

We are using Nextflow with nf-core, and nf-core downloads software using Git and GitHub. Therefore, we have to enable access to GitHub to ensure that we can fetch the right tools when using Nextflow. To do this, we have to create what's called a **scm** file (no file extension!) in our nextflow folder. This file specifies the account to use with the Nextflow software and its respective credentials. We generate this file as follows: 

```
providers {

  github {
    user = 'me'
    password = 'my-personal-access-token'
  }

}
```

**Very important:** Once this file is created, we have to move it to the following location: **$HOME/.nextflow/scm** <br>  
To do this, type ```mv scm $HOME/.nextflow/scm```

Once this has been set up, we are ready to get to the last step and start using Nextflow on OSCAR! 

### Step 4: Selecting An nf-core Pipeline

nf-core has many analysis pipelines we can use and so we need to identify the specific pipeline that is appropriate for our needs. We are once again using the RRBS example that we started with, so we need a pipeline that is appropriate to use for processing RRBS data. Heading over to https://nf-co.re/pipelines we can see that the **methylseq** pipeline will work for our data. We can view the details of this pipeline, such as all of its arguments and the steps it performs, by cliking on its link or visiting here: https://nf-co.re/methylseq 

**Note:** Visitng the pipeline's page and reviewing the pipeline's documentation is important because we need to know what arguments to use to make it run correctly. Once we've reviewed this and got an idea of how we need to run things, we can move to the final step. 

### Step 4: Shell Script To Bring It All Together

The last step is to create a shell script to run Nextflow with the desired nf-core pipeline on OSCAR. To do this, we create a bash script in our nextflow folder called **nextflow.sh**. This script is written as follows: 

**nextflow.sh**

```
#!/bin/bash

###############################
#                             #
#   Nextflow for BootCamp     #
#                             #
###############################

##### 1.) Job Sumbission Options ######

# Can change/add resources as needed 

#SBATCH -n 4
#SBATCH --mem 8G
#SBATCH -t 2:00:00

# Logs
#SBATCH --mail-user=jordan_lawson@brown.edu  
#SBATCH --mail-type=ALL
#SBATCH --output=%x-%j.out

##### 2.) Run Nextflow #####

# Activate environment previosuly created 
source /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/nextflow/nextflow_env/bin/activate

# Run nextflow 
nextflow run nf-core/methylseq -profile singularity --input "/gpfs/data/ccvstaff/jlawson9/bootcamp_2022/data_new/*.fq.gz" --single_end --genome Sscrofa10.2 -c /gpfs/data/ccvstaff/jlawson9/bootcamp_2022/nextflow/nextflow.config --outdir $HOME/scratch
```

Once we have this file with everything else, we can run everything on the cluster by typing ```sbatch nextflow.sh```

Note in the above that we are running the nextflow program with the methylseq pipeline, as specified in the ```nf-core/methylseq``` argument that comes right after the ```nextflow run``` command. This tells nextflow to fetch and download the methylseq pipeline from nf-core. Some more important notes: 

- your log file for slurm will automtatically be placed in the same directory that you ran the nextflow.sh script from. You can control where log files go using the ```-log``` flag followed by the path you wish to store the logs at. 

- the -c flag handles the nextflow.config file we created

- we are using the -profile singularity argument in the above shell script so that Nextflow uses a singularity container when the running pipeline; this is highly recommended for reproducibility. 

- the other arguments are bioinformatics specific and are there to make sure the pipeline runs according to our needs; they were found by visitng the pipeline's documentation page at: https://nf-co.re/methylseq (which you can get to from the nf-core homepage). 

## A Few Closing Thoughts.....

This workshop just provided an introductory overview of running worklfow management tools on OSCAR. There is much more customization that you can do and much more advanced things you can perform. Some of these are: 

* Resource allocation by rule (or analysis step) 
* Using a config.yaml file to handle your samples and how you iterate through files
* Running pipelines that skip steps and start at a specific step (for example, pipelines that fail at a certain step, you may not want to repeat everything but instead pick up where you left off)
* Handling log files within specific steps so that you get detailed output for each rule 
* And much more....